# Sphere

In [10]:
import numpy as np
from book3.dezero import Variable

def sphere(x, y):
    z = x ** 2 + y ** 2
    return z

x = Variable(np.array(1.0))
y = Variable(np.array(1.0))
z = sphere(x, y)
z.backward()
print(x.grad, y.grad)

2.0 2.0



# matyas

In [11]:
"""
def matyas(x, y):
    z = sub(mul(0.26), add(pow(x, 2), pow(y, 2))), mul(0.48, mul(x, y)))
    return z
"""
def matyas(x, y):
    z = 0.26 * (x ** 2 + y ** 2) - 0.48 * x * y
    return z

x = Variable(np.array(1.0))
y = Variable(np.array(1.0))
z = matyas(x, y)
z.backward()
print(x.grad, y.grad)

0.040000000000000036 0.040000000000000036


# Goldstein-Price 함수

In [17]:
def goldstein(x, y):
    z = (1 + (x + y + 1)**2 * (19 - 14*x + 3*x**2 - 14*y + 6*x*y + 3*y**2)) * \
        (30 + (2*x - 3*y)**2 * (18 - 32*x + 12*x**2 + 48*y - 36*x*y + 27*y**2))
    return z

x = Variable(np.array(1.0))
y = Variable(np.array(1.0))
z = goldstein(x, y)
z.backward()
print(x.grad, y.grad)

-5376.0 8064.0


In [54]:
import unittest

def goldstein(x, y):
    z = (1 + (x + y + 1)**2 * (19 - 14*x + 3*x**2 - 14*y + 6*x*y + 3*y**2)) * \
        (30 + (2*x - 3*y)**2 * (18 - 32*x + 12*x**2 + 48*y - 36*x*y + 27*y**2))
    return z


def numerical_diff(f, x, y, eps=1e-4):
    # 두 변수에 대한 수치 미분 계산
    # x와 y의 각각에 대해 작은 변화량을 주어 미분값을 계산
    x0 = Variable(x.data - eps)
    x1 = Variable(x.data + eps)
    y0 = Variable(y.data - eps)
    y1 = Variable(y.data + eps)

    # 함수 계산
    fx0y = f(x0, y)  # (x - eps, y - eps)
    fx1y = f(x1, y)  # (x + eps, y - eps)
    fxy0 = f(x, y0)  # (x - eps, y + eps)
    fxy1 = f(x, y1)  # (x + eps, y + eps)

    # x와 y에 대한 기울기 계산
    grad_x = (fx1y - fx0y) / (2 * eps)  # x에 대한 기울기
    grad_y = (fxy1 - fxy0) / (2 * eps)  # y에 대한 기울기

    return grad_x, grad_y

def numerical_diff2(f, x):
    h = 1e-4  # 0.0001
    grad = np.zeros_like(x)

    for idx in range(x.size):
        tmp_val = x[idx]
        x[idx] = tmp_val + h
        fxh1 = f(*x)  # f(x+h)

        x[idx] = tmp_val - h
        fxh2 = f(*x)  # f(x-h)
        grad[idx] = (fxh1 - fxh2) / (2 * h)

        x[idx] = tmp_val  # 값 복원

    return grad


class SquareTest(unittest.TestCase):
    def test_gradient_check(self):
        x = Variable(np.random.rand(1)) # 무작위 입력값 생성
        y = Variable(np.random.rand(1)) # 무작위 입력값 생성
        z = goldstein(x, y)
        z.backward()
        # num_grad = numerical_diff(goldstein, x, y)
        num_grad = numerical_diff2(goldstein, np.array([x, y]))
        flg = np.allclose(x.grad.data, num_grad[0].data) and \
              np.allclose(y.grad.data, num_grad[1].data)
        self.assertTrue(flg)

unittest.main(argv=[''], verbosity=2, exit=False)

test_gradient_check (__main__.SquareTest) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.004s

OK
